# MCP Server

## 첫 MCP 서버 만들기

먼저 간단한 계산기 기능을 가진 데모 서버를 생성한다. 

`FastMCP`를 사용하여 서버를 만들고, 두 숫자를 더하는 `add`라는 간단한 도구를 추가한 것이다.

In [ ]:
# 파일명: server.py

from mcp.server.fastmcp import FastMCP
from dotenv import load_dotenv

load_dotenv("../.env")

# MCP 서버 생성
mcp = FastMCP(
    name="Calculator",
    host="0.0.0.0",  # SSE 전송 방식에서만 사용됨 (localhost)
    port=8050,  # SSE 전송 방식에서만 사용됨 (원하는 포트로 설정)
    stateless_http=True,
)


# 간단한 계산기 도구 추가
@mcp.tool()
def add(a: int, b: int) -> int:
    """두 숫자를 더한다"""
    return a + b


# 서버 실행
if __name__ == "__main__":
    # 사용할 전송 방식을 선택한다.
    # transport = "stdio"
    # transport = "sse"
    transport = "streamable-http"
    
    if transport == "stdio":
        print("stdio 전송 방식으로 서버를 실행한다")
        mcp.run(transport="stdio")
    elif transport == "sse":
        print("SSE 전송 방식으로 서버를 실행한다")
        mcp.run(transport="sse")
    elif transport == "streamable-http":
        print("Streamable HTTP 전송 방식으로 서버를 실행한다")
        mcp.run(transport="streamable-http")
    else:
        raise ValueError(f"알 수 없는 전송 방식: {transport}")

## 서버 실행하기

#### 1. MCP Inspector를 사용한 개발 모드

서버를 테스트하는 가장 쉬운 방법은 MCP Inspector를 사용하는 것이다.
```python
mcp dev server.py
```
이 명령어는 서버를 로컬에서 실행하고 MCP Inspector(웹 기반 도구)에 연결하여 서버의 도구와 리소스를 직접 테스트해 볼 수 있게 한다.

#### 2. Claude 데스크톱 통합

Claude 데스크톱이 설치되어 있다면, 다음과 같이 서버를 설치하여 Claude와 함께 사용할 수 있다.
```python
mcp install server.py
```

#### 3. 직접 실행

서버를 직접 실행할 수도 있다.
```python
# 방법 1: 파이썬 스크립트로 실행
python server.py

# 방법 2: UV 사용 (권장)
uv run server.py
```

### MCP 서버는 어떻게 실행되는가?

MCP 서버를 실행하면 다음과 같은 일이 일어난다:
1. 서버가 정의된 기능(도구, 리소스 등)으로 초기화된다.
2. 특정 전송 방식을 통해 연결을 기다리기 시작한다.

기본적으로 MCP 서버는 전통적인 웹 서버 포트를 사용하지 않고 다음 방식 중 하나를 사용한다:
- **stdio 전송**: 서버가 표준 입출력을 통해 통신한다 (`mcp run`의 기본값 및 Claude 데스크톱 통합에 사용).
- **SSE 전송**: HTTP 기반 통신을 위해 사용된다 (명시적으로 설정 시).
- **Streamable HTTP 전송**: 고성능 및 확장성을 위한 최신 HTTP 스트리밍 기반 통신 방식이다.

## 클라이언트 구현


### 표준 입출력(Standard I/O) 클라이언트

클라이언트와 서버가 같은 프로세스에서 실행되거나 클라이언트가 서버 프로세스를 직접 시작하는 경우에 `stdio` 방식을 사용한다. 이 클라이언트는 `python server.py` 명령으로 서버 프로세스를 시작하고 표준 입출력을 통해 통신한다.

In [ ]:
# 파일명: client-stdio.py

import asyncio
import nest_asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

nest_asyncio.apply()  # 대화형 파이썬 실행에 필요하다


async def main():
    # 서버 파라미터 정의
    server_params = StdioServerParameters(
        command="python",  # 서버 실행 명령어
        args=["server.py"],  # 명령어에 전달할 인자
    )

    # 서버에 연결
    async with stdio_client(server_params) as (read_stream, write_stream):
        async with ClientSession(read_stream, write_stream) as session:
            # 연결 초기화
            await session.initialize()

            # 사용 가능한 도구 목록 조회
            tools_result = await session.list_tools()
            print("사용 가능한 도구:")
            for tool in tools_result.tools:
                print(f"  - {tool.name}: {tool.description}")

            # 계산기 도구 호출
            result = await session.call_tool("add", arguments={"a": 2, "b": 3})
            print(f"2 + 3 = {result.content[0].text}")


if __name__ == "__main__":
    # 위 server.py 코드에서 transport를 "stdio"로 설정하고 실행해야 한다.
    asyncio.run(main())

### 서버-전송 이벤트(SSE) 클라이언트

HTTP를 통해 서버와 통신하려면 `server.py`에서 SSE 전송 방식을 활성화해야 한다. 이 클라이언트는 `http://localhost:8050/sse` 엔드포인트로 서버에 연결한다.

In [ ]:
# 파일명: client-sse.py

import asyncio
import nest_asyncio
from mcp import ClientSession
from mcp.client.sse import sse_client

nest_asyncio.apply()  # 대화형 파이썬 실행에 필요하다

"""
이 스크립트를 실행하기 전 확인 사항:
1. 서버가 실행 중이어야 한다.
2. 서버가 SSE 전송 방식을 사용하도록 설정되어야 한다.
3. 서버가 8050 포트에서 수신 대기 중이어야 한다.

서버 실행 명령어:
uv run server.py
"""


async def main():
    # SSE를 사용하여 서버에 연결
    async with sse_client("http://localhost:8050/sse") as (read_stream, write_stream):
        async with ClientSession(read_stream, write_stream) as session:
            # 연결 초기화
            await session.initialize()

            # 사용 가능한 도구 목록 조회
            tools_result = await session.list_tools()
            print("사용 가능한 도구:")
            for tool in tools_result.tools:
                print(f"  - {tool.name}: {tool.description}")

            # 계산기 도구 호출
            result = await session.call_tool("add", arguments={"a": 2, "b": 3})
            print(f"2 + 3 = {result.content[0].text}")


if __name__ == "__main__":
    # 위 server.py 코드에서 transport를 "sse"로 설정하고 실행해야 한다.
    asyncio.run(main())

### 스트림 가능 HTTP(Streamable HTTP) 클라이언트 - **NEW**

> **참고**: Streamable HTTP 전송 방식은 **2025년 3월 24일**에 도입되었으며, 현재 SSE 전송 방식을 대체하는 **운영 배포 환경의 권장 방식**이다.

**Streamable HTTP를 사용하는 이유**

Streamable HTTP는 SSE에 비해 여러 장점을 제공한다:
- **더 나은 성능**: 높은 동시성 환경에서 3-5배의 성능 향상이 있다.
- **단순화된 아키텍처**: 별도의 HTTP + SSE 엔드포인트 대신 단일 엔드포인트를 사용한다.
- **향상된 확장성**: 다중 노드 배포를 더 잘 지원한다.
- **최신 표준**: 현재의 HTTP 스트리밍 표준을 기반으로 구축되었다.

In [ ]:
# 파일명: client-streamable-http.py

import asyncio
import nest_asyncio
from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

nest_asyncio.apply()  # 대화형 파이썬 실행에 필요하다

"""
이 스크립트를 실행하기 전 확인 사항:
1. 서버가 실행 중이어야 한다.
2. 서버가 streamable-http 전송 방식을 사용하도록 설정되어야 한다.
3. 서버가 8050 포트에서 수신 대기 중이어야 한다.

서버 실행 명령어:
uv run server.py
"""


async def main():
    # Streamable HTTP를 사용하여 서버에 연결
    async with streamablehttp_client("http://localhost:8050/mcp") as (
        read_stream,
        write_stream,
        get_session_id,
    ):
        async with ClientSession(read_stream, write_stream) as session:
            # 연결 초기화
            await session.initialize()

            # 사용 가능한 도구 목록 조회
            tools_result = await session.list_tools()
            print("사용 가능한 도구:")
            for tool in tools_result.tools:
                print(f"  - {tool.name}: {tool.description}")

            # 계산기 도구 호출
            result = await session.call_tool("add", arguments={"a": 2, "b": 3})
            print(f"2 + 3 = {result.content[0].text}")


if __name__ == "__main__":
    # 위 server.py 코드에서 transport를 "streamable-http"로 설정하고 실행해야 한다.
    asyncio.run(main())

## 어떤 접근 방식을 선택해야 하는가?

- **`stdio` 사용**: 클라이언트와 서버가 같은 프로세스에서 실행되거나 클라이언트에서 직접 서버 프로세스를 시작하는 경우에 적합하다.
- **`Streamable HTTP` 사용**: 최고의 성능과 확장성이 필요한 운영 배포 환경에 권장된다.
- **`SSE` 사용**: 개발 중이거나, Streamable HTTP를 아직 지원하지 않는 구형 MCP 구현과 작업할 때 사용한다.

대부분의 운영 백엔드 통합의 경우, **Streamable HTTP** 방식이 최고의 성능과 현대적인 아키텍처를 제공하며, `stdio`는 개발이나 긴밀하게 결합된 시스템에 더 간단한 대안이 될 수 있다.